## RECOMMENDER - PHASE 2

**SUMMARY:**

The following model attempts to structure a recommender that adds the number of actual ratings per observation into the evaluation.  This would buffer against a high rating for a single wine being posted by an individual, or a wine with a low rating being elevated by rating count.  The beginning of this process starts <a href='#RECOMMENDER 2'>here.</a>

#### IMPORT PACKAGES

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

#### IMPORT WINE CATALOG AND USER FILES AND PREP FOR PIVOT TABLE

In [3]:
wine = pd.read_csv('./data/wine_catalog.csv')
wine = wine.fillna("")
wine.head()

,Unnamed: 0,winery,wine,rating,wine_style,wine_category,vintage,region,country,price,contact_for_price,wine_style_code,wine_category_code,region_code,country_code
0,0,Caymus,Cabernet Sauvignon N.V.,4.6,Cabernet Sauvignon,Red,N.V.,Napa Valley,United States,79.05,,15,2,587,36
1,1,The Prisoner,The Prisoner N.V.,4.4,Blend,Red,N.V.,Napa Valley,United States,39.75,,10,2,587,36
2,2,Masi,Costasera Amarone della Valpolicella Classico ...,4.3,Valpolicella,Red,N.V.,Amarone della Valpolicella Classico,Italy,54.99,,86,2,34,19
3,3,Stag's Leap Wine Cellars,ARTEMIS Cabernet Sauvignon N.V.,4.3,Cabernet Sauvignon,Red,N.V.,Napa Valley,United States,59.99,,15,2,587,36
4,4,Banfi,Brunello di Montalcino N.V.,4.2,Brunello,Red,N.V.,Brunello di Montalcino,Italy,65.98,,12,2,120,19


In [4]:
wine.drop(['Unnamed: 0','winery','wine_style','wine_category','vintage','region','country','price','contact_for_price','wine_style_code','wine_category_code','region_code','country_code'],axis=1,inplace=True)

In [5]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33163 entries, 0 to 33162
Data columns (total 2 columns):
wine      33163 non-null object
rating    33163 non-null float64
dtypes: float64(1), object(1)
memory usage: 518.3+ KB


In [6]:
wine=wine.rename(columns = {'wine':'wine_label'})
wine.head()

,wine_label,rating
0,Cabernet Sauvignon N.V.,4.6
1,The Prisoner N.V.,4.4
2,Costasera Amarone della Valpolicella Classico ...,4.3
3,ARTEMIS Cabernet Sauvignon N.V.,4.3
4,Brunello di Montalcino N.V.,4.2


In [7]:
ratings = pd.read_csv('./data/user.csv')
#ratings = ratings.fillna(0.0)
ratings.head()

,Unnamed: 0,user-0,user-1,user-2,user-3,user-4,user-5,user-6,user-7,user-8,...,user-990,user-991,user-992,user-993,user-994,user-995,user-996,user-997,user-998,user-999
0,0,NaN,NaN,4.7,4.6,4.9,4.8,NaN,4.3,NaN,...,NaN,NaN,NaN,4.8,4.4,4.7,NaN,4.8,NaN,4.6
1,1,4.5,4.3,NaN,NaN,NaN,NaN,4.4,NaN,4.2,...,NaN,NaN,NaN,NaN,4.1,NaN,4.4,4.1,4.2,4.3
2,2,NaN,4.1,NaN,NaN,4.2,4.6,NaN,NaN,NaN,...,4.3,4.2,NaN,4.3,NaN,4.5,4.0,NaN,NaN,NaN
3,3,NaN,NaN,4.5,NaN,NaN,4.3,NaN,NaN,4.5,...,NaN,4.2,4.1,4.3,4.6,4.2,4.5,NaN,4.2,NaN
4,4,4.3,4.2,NaN,4.4,4.1,4.2,NaN,NaN,NaN,...,4.4,NaN,NaN,4.1,NaN,4.5,4.3,4.0,NaN,4.2


In [8]:
ratings.drop(['Unnamed: 0'],axis=1, inplace=True)

In [9]:
wine.insert(0,'wine_id',wine.index+1)
ratings.insert(0,'wine_id',ratings.index+1)

In [10]:
ratings

,wine_id,user-0,user-1,user-2,user-3,user-4,user-5,user-6,user-7,user-8,...,user-990,user-991,user-992,user-993,user-994,user-995,user-996,user-997,user-998,user-999
0,1,NaN,NaN,4.7,4.6,4.9,4.8,NaN,4.3,NaN,...,NaN,NaN,NaN,4.8,4.4,4.7,NaN,4.8,NaN,4.6
1,2,4.5,4.3,NaN,NaN,NaN,NaN,4.4,NaN,4.2,...,NaN,NaN,NaN,NaN,4.1,NaN,4.4,4.1,4.2,4.3
2,3,NaN,4.1,NaN,NaN,4.2,4.6,NaN,NaN,NaN,...,4.3,4.2,NaN,4.3,NaN,4.5,4.0,NaN,NaN,NaN
3,4,NaN,NaN,4.5,NaN,NaN,4.3,NaN,NaN,4.5,...,NaN,4.2,4.1,4.3,4.6,4.2,4.5,NaN,4.2,NaN
4,5,4.3,4.2,NaN,4.4,4.1,4.2,NaN,NaN,NaN,...,4.4,NaN,NaN,4.1,NaN,4.5,4.3,4.0,NaN,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33158,33159,3.8,NaN,3.8,NaN,3.7,NaN,NaN,NaN,3.6,...,3.5,3.7,3.8,3.8,3.5,NaN,3.9,3.3,3.4,3.9
33159,33160,3.3,3.9,3.8,NaN,NaN,NaN,3.8,NaN,NaN,...,3.3,NaN,NaN,3.3,NaN,NaN,NaN,NaN,3.4,NaN
33160,33161,NaN,3.9,NaN,NaN,NaN,NaN,NaN,3.7,NaN,...,NaN,3.7,3.4,NaN,NaN,NaN,NaN,3.9,3.9,NaN
33161,33162,NaN,NaN,NaN,3.4,3.7,3.7,NaN,3.7,NaN,...,3.2,NaN,3.8,NaN,NaN,3.6,NaN,3.8,NaN,NaN


In [11]:
ratings.to_csv('./data/ratings.csv')

In [12]:
#going to use pd.melt() to to massage both DataFrames into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value.
wine_1 = pd.melt(wine,id_vars=['wine_label'],var_name="wine_style",value_name="wine_id")

In [50]:
wine_1.head(10)

,wine_label,wine_style,wine_id
0,Cabernet Sauvignon N.V.,wine_id,1.0
1,The Prisoner N.V.,wine_id,2.0
2,Costasera Amarone della Valpolicella Classico ...,wine_id,3.0
3,ARTEMIS Cabernet Sauvignon N.V.,wine_id,4.0
4,Brunello di Montalcino N.V.,wine_id,5.0
5,Opus One N.V.,wine_id,6.0
6,Valbuena 5º N.V.,wine_id,7.0
7,Rouge (Gaston Hochar) N.V.,wine_id,8.0
8,Ribera del Duero N.V.,wine_id,9.0
9,Napa Valley Cabernet Sauvignon N.V.,wine_id,10.0


In [14]:
ratings_1=pd.melt(ratings,id_vars=['wine_id'], var_name='user',value_name="rating")

In [15]:
ratings_1.head()

,wine_id,user,rating
0,1,user-0,NaN
1,2,user-0,4.5
2,3,user-0,NaN
3,4,user-0,NaN
4,5,user-0,4.3


#### CONSTRUCT THE PIVOT TABLE

In [16]:
reco_1 = pd.merge(wine_1, ratings_1, on='wine_id')
reco_1.shape

C:\Users\tonyb\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1089: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


(36944000, 5)

In [17]:
reco_1.head()

,wine_label,wine_style,wine_id,user,rating
0,Cabernet Sauvignon N.V.,wine_id,1.0,user-0,NaN
1,Cabernet Sauvignon N.V.,wine_id,1.0,user-1,NaN
2,Cabernet Sauvignon N.V.,wine_id,1.0,user-2,4.7
3,Cabernet Sauvignon N.V.,wine_id,1.0,user-3,4.6
4,Cabernet Sauvignon N.V.,wine_id,1.0,user-4,4.9


In [18]:
pivot = pd.pivot_table(reco_1, index='wine_label', columns='user', values='rating')
pivot.head()

user,user-0,user-1,user-10,user-100,user-101,user-102,user-103,user-104,user-105,user-106,...,user-990,user-991,user-992,user-993,user-994,user-995,user-996,user-997,user-998,user-999
wine_label,,,,,,,,,,,,,,,,,,,,,
#42 Meritage Red Blend 2016,3.30,NaN,3.800000,3.400000,NaN,3.60,NaN,3.500,3.70,NaN,...,3.600000,NaN,3.70,3.600,3.90,NaN,3.900000,3.6,NaN,NaN
'61 Franciacorta Nature 2010,3.85,4.300000,4.371429,4.033333,4.05,4.22,4.183333,4.100,4.25,4.085714,...,3.700000,4.083333,4.10,4.125,4.44,4.10,4.283333,4.1,4.150,3.700
'Giulio Ferrari' Riserva del Fondatore 1993,4.90,4.575000,4.633333,4.475000,4.30,4.60,4.300000,4.650,NaN,4.600000,...,4.700000,4.700000,4.40,4.450,4.45,4.90,4.533333,4.6,4.625,4.850
'Giulio Ferrari' Riserva del Fondatore 1997,4.90,NaN,4.550000,4.533333,4.45,NaN,4.900000,4.700,4.30,4.500000,...,4.633333,4.666667,4.75,4.650,4.60,4.85,4.650000,NaN,4.600,4.600
'Giulio Ferrari' Riserva del Fondatore 2004,4.50,4.466667,4.200000,4.600000,4.70,4.45,4.475000,4.625,4.50,4.400000,...,4.700000,4.650000,4.40,4.750,4.45,4.45,4.700000,4.6,4.400,4.375


In [19]:
pivot.shape

(10450, 1000)

#### CREATE THE SPARSE MATRIX

In [20]:
sparse_pivot=sparse.csr_matrix(pivot.fillna(0.0))

In [21]:
print(sparse_pivot)

  (0, 0)	3.3
  (0, 2)	3.8
  (0, 3)	3.4
  (0, 5)	3.6
  (0, 7)	3.5
  (0, 8)	3.7
  (0, 11)	3.7
  (0, 12)	3.5
  (0, 14)	3.3
  (0, 15)	3.9
  (0, 16)	3.6
  (0, 18)	3.4
  (0, 19)	3.5
  (0, 20)	3.7
  (0, 27)	3.4
  (0, 31)	3.5
  (0, 33)	3.3
  (0, 36)	3.3
  (0, 39)	3.4
  (0, 40)	3.4
  (0, 41)	3.5
  (0, 43)	3.4
  (0, 44)	3.3
  (0, 47)	3.6
  (0, 48)	3.4
  :	:
  (10449, 975)	3.95
  (10449, 976)	3.8
  (10449, 977)	3.7
  (10449, 978)	3.8499999999999996
  (10449, 979)	3.6500000000000004
  (10449, 980)	3.6500000000000004
  (10449, 981)	3.9333333333333336
  (10449, 982)	3.8
  (10449, 983)	4.066666666666666
  (10449, 984)	4.05
  (10449, 985)	4.1
  (10449, 986)	3.9
  (10449, 987)	3.8
  (10449, 988)	3.8499999999999996
  (10449, 989)	3.6500000000000004
  (10449, 990)	4.066666666666666
  (10449, 991)	3.8666666666666667
  (10449, 992)	4.2
  (10449, 993)	4.1
  (10449, 994)	3.9749999999999996
  (10449, 995)	3.75
  (10449, 996)	3.7
  (10449, 997)	4.2
  (10449, 998)	3.8333333333333335
  (10449, 999)	3.90000000000

#### CALCULATE PAIRWISE DISTANCES

In [22]:
recommender = pairwise_distances(sparse_pivot, metric='cosine')

#### BUILD THE RECOMMENDER

In [23]:
recommender_df = pd.DataFrame(recommender, columns=pivot.index, index=pivot.index)
recommender_df.head()

wine_label,#42 Meritage Red Blend 2016,'61 Franciacorta Nature 2010,'Giulio Ferrari' Riserva del Fondatore 1993,'Giulio Ferrari' Riserva del Fondatore 1997,'Giulio Ferrari' Riserva del Fondatore 2004,'Giulio Ferrari' Riserva del Fondatore 2005,'Giulio Ferrari' Riserva del Fondatore 2006,'Giulio Ferrari' Riserva del Fondatore 2007,'Rosso & Bianco' Bianco (Pinot Grigio) 2017,(B) Old Vine Zinfandel Lodi 2016,...,Ürziger Würzgarten Riesling Spätlese 2018,Ürziger Würzgarten Riesling Spätlese N.V.,ΓΙΟΡΤΗ (Feast) White 2018,Ξυνιστέρι (Xynisteri) 2018,ΩΔΗ ΠΑΝΟΣ (Ode Panos) N.V.,‘61 Franciacorta Brut 1961,‘61 Franciacorta Brut N.V.,‘61 Franciacorta Rosé 1961,‘61 Franciacorta Rosé N.V.,‘61 Franciacorta Satèn 1961
wine_label,,,,,,,,,,,,,,,,,,,,,
#42 Meritage Red Blend 2016,0.000000,0.307243,0.320212,0.313042,0.319982,0.311664,0.315161,0.316054,0.473012,0.487344,...,0.489250,0.346001,0.500101,0.493689,0.314372,0.314413,0.322874,0.333469,0.311157,0.311883
'61 Franciacorta Nature 2010,0.307243,0.000000,0.048162,0.047457,0.049482,0.046418,0.040357,0.046592,0.282304,0.295366,...,0.319757,0.079676,0.296630,0.310398,0.051346,0.049843,0.051451,0.058169,0.044640,0.045102
'Giulio Ferrari' Riserva del Fondatore 1993,0.320212,0.048162,0.000000,0.063390,0.062203,0.060993,0.062067,0.063703,0.289916,0.314535,...,0.328691,0.090053,0.309575,0.324033,0.067168,0.066136,0.068959,0.076833,0.059594,0.060693
'Giulio Ferrari' Riserva del Fondatore 1997,0.313042,0.047457,0.063390,0.000000,0.062010,0.060340,0.056132,0.058057,0.296826,0.310949,...,0.318012,0.091314,0.306676,0.320409,0.060469,0.061023,0.064917,0.070314,0.057315,0.060831
'Giulio Ferrari' Riserva del Fondatore 2004,0.319982,0.049482,0.062203,0.062010,0.000000,0.063007,0.062385,0.061310,0.291377,0.309192,...,0.327039,0.093383,0.314748,0.317847,0.065696,0.065423,0.063302,0.075994,0.059793,0.062109


In [24]:
## TEST THE RECOMMENDER

In [25]:
search = 'Rosso & Bianco'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 3.595158286778398
Number of ratings 537

10 closest wine labels

wine_label
Red Blend 2015                      0.266419
Sparkling N.V.                      0.266593
Crémant de Limoux Brut N.V.         0.266731
Sparkling Brut N.V.                 0.266846
Brut Champagne 2000                 0.267086
Brut Premier Champagne N.V.         0.267143
Extra Brut Champagne N.V.           0.267214
Pauillac (Grand Cru Classé) 2014    0.267254
Brut Champagne 1er Cru N.V.         0.267284
Châteauneuf-du-Pape 2014            0.267308
Name: 'Rosso & Bianco' Bianco (Pinot Grigio) 2017, dtype: float64

*******************************************************************************************



In [26]:
search = 'Reserva Tempranillo 2014'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 3.7071428571428635
Number of ratings 532

10 closest wine labels

wine_label
Brut Rosé Champagne 2004                0.270355
Franciacorta Rosé N.V.                  0.270846
Sauvignon Blanc 2015                    0.270858
Langhe Rosso N.V.                       0.270869
Extra Brut N.V.                         0.270870
Margaux (Grand Cru Classé) 2009         0.271042
Saint-Julien (Grand Cru Classé) 2009    0.271081
Bella Vigna Chardonnay N.V.             0.271108
Saint-Julien (Grand Cru Classé) 2006    0.271120
Bianco di Ampeleia 2018                 0.271128
Name: Reserva Tempranillo 2014, dtype: float64

*******************************************************************************************



<a id='RECOMMENDER 2'></a>

####  RECOMMENDER 2: BASED ON RATING COUNTS - PEARSON R CORRELATION

In [107]:
pd.set_option('display.max_columns',None)

In [28]:
rating_counts = pd.read_csv('./data/rating_counts.csv')

In [29]:
rating_counts.shape

(33163, 2)

In [30]:
rating_counts.drop(['Unnamed: 0'],axis=1, inplace=True)
rating_counts.insert(0,'wine_id',rating_counts.index+1)

In [31]:
rating_counts

,wine_id,rating_counts
0,1,489
1,2,493
2,3,519
3,4,487
4,5,517
...,...,...
33158,33159,478
33159,33160,488
33160,33161,512
33161,33162,485


In [108]:
user_data=pd.read_csv('./data/user.csv')
user_data.drop(['Unnamed: 0'],axis=1, inplace=True)
user_data

user-0  user-1  user-2  user-3  user-4  user-5  user-6  user-7  user-8  \
0         NaN     NaN     4.7     4.6     4.9     4.8     NaN     4.3     NaN   
1         4.5     4.3     NaN     NaN     NaN     NaN     4.4     NaN     4.2   
2         NaN     4.1     NaN     NaN     4.2     4.6     NaN     NaN     NaN   
3         NaN     NaN     4.5     NaN     NaN     4.3     NaN     NaN     4.5   
4         4.3     4.2     NaN     4.4     4.1     4.2     NaN     NaN     NaN   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
33158     3.8     NaN     3.8     NaN     3.7     NaN     NaN     NaN     3.6   
33159     3.3     3.9     3.8     NaN     NaN     NaN     3.8     NaN     NaN   
33160     NaN     3.9     NaN     NaN     NaN     NaN     NaN     3.7     NaN   
33161     NaN     NaN     NaN     3.4     3.7     3.7     NaN     3.7     NaN   
33162     NaN     NaN     3.7     3.2     3.7     NaN     NaN     NaN     NaN   

       user-9  user-10  user-11  user-12  user-13  user-14  user-15  user-16  \
0         NaN      NaN      4.8      NaN      NaN      NaN      NaN      NaN   
1         NaN      NaN      NaN      4.6      4.2      NaN      NaN      4.4   
2         NaN      NaN      NaN      NaN      NaN      NaN      4.0      NaN   
3         NaN      4.6      NaN      NaN      4.3      4.6      4.0      4.4   
4         3.9      4.2      NaN      4.0      NaN      4.3      NaN      NaN   
...       ...      ...      ...      ...      ...      ...      ...      ...   
33158     3.3      NaN      3.8      NaN      NaN      3.3      3.5      3.9   
33159     NaN      3.9      NaN      NaN      NaN      NaN      NaN      NaN   
33160     NaN      3.6      NaN      3.3      NaN      NaN      3.8      3.4   
33161     NaN      NaN      NaN      3.2      NaN      3.5      3.6      3.6   
33162     NaN      3.8      NaN      NaN      NaN      NaN      3.4      3.2   

       user-17  user-18  user-19  user-20  user-21  user-22  user-23  user-24  \
0          NaN      NaN      4.7      4.4      NaN      4.6      4.9      4.4   
1          NaN      NaN      NaN      4.7      4.4      4.1      NaN      NaN   
2          4.6      NaN      NaN      4.4      NaN      4.3      NaN      NaN   
3          NaN      4.3      NaN      NaN      NaN      4.3      NaN      4.5   
4          4.1      4.5      NaN      NaN      4.2      NaN      3.9      3.9   
...        ...      ...      ...      ...      ...      ...      ...      ...   
33158      NaN      NaN      NaN      3.7      NaN      3.3      NaN      NaN   
33159      NaN      3.7      NaN      NaN      NaN      NaN      NaN      NaN   
33160      NaN      NaN      NaN      NaN      3.3      NaN      3.3      NaN   
33161      3.5      NaN      NaN      NaN      3.4      NaN      3.4      3.6   
33162      NaN      3.3      NaN      NaN      NaN      3.5      3.5      NaN   

       user-25  user-26  user-27  user-28  user-29  user-30  user-31  user-32  \
0          4.5      NaN      4.8      NaN      NaN      4.8      NaN      NaN   
1          4.7      4.6      NaN      NaN      NaN      NaN      4.1      NaN   
2          4.4      4.0      4.5      4.4      4.1      NaN      4.0      NaN   
3          NaN      NaN      NaN      NaN      NaN      NaN      4.3      NaN   
4          NaN      4.5      4.1      4.4      4.2      NaN      4.1      3.9   
...        ...      ...      ...      ...      ...      ...      ...      ...   
33158      3.7      3.4      3.6      3.6      NaN      NaN      NaN      3.6   
33159      NaN      3.4      NaN      NaN      3.7      3.5      NaN      3.5   
33160      3.7      NaN      NaN      NaN      3.4      NaN      NaN      NaN   
33161      3.6      NaN      NaN      3.5      NaN      3.4      NaN      3.2   
33162      NaN      3.4      NaN      3.3      NaN      3.6      NaN      NaN   

       user-33  user-34  user-35  user-36  user-37  user-38  user-39  user-40  \
0          4.6      NaN      NaN      NaN     

In [33]:
rating_counts['avg_rating']=pd.DataFrame(user_data.mean(axis=1))

In [34]:
rating_counts

,wine_id,rating_counts,avg_rating
0,1,489,4.607566
1,2,493,4.399594
2,3,519,4.284200
3,4,487,4.312320
4,5,517,4.200387
...,...,...,...
33158,33159,478,3.603766
33159,33160,488,3.606557
33160,33161,512,3.616602
33161,33162,485,3.491753


In [35]:
# This gives us the most rated wine in the dataset
# We can see that the wine with the highest rating count is not necessarily the one with the highest rating
rating_counts.sort_values('rating_counts',ascending=False).head()

,wine_id,rating_counts,avg_rating
15044,15045,568,3.977113
5688,5689,562,4.600356
2555,2556,560,4.200000
5441,5442,560,4.598750
1794,1795,557,4.601616


In [78]:
ratings_pivot = ratings_1.pivot(index='user', columns='wine_id')
user = ratings_pivot.index
wine_id = ratings_pivot.columns
print(ratings_pivot.shape)
ratings_pivot.head()

(1000, 33163)


rating                                                        ...  \
wine_id   1     2     3     4     5     6     7     8     9     10     ...   
user                                                                   ...   
user-0      NaN   4.5   NaN   NaN   4.3   4.8   4.5   3.9   4.5   4.7  ...   
user-1      NaN   4.3   4.1   NaN   4.2   NaN   4.7   NaN   4.7   NaN  ...   
user-10     NaN   NaN   NaN   4.6   4.2   NaN   NaN   4.5   4.3   4.8  ...   
user-100    NaN   4.4   4.4   NaN   4.4   NaN   NaN   NaN   4.2   NaN  ...   
user-101    NaN   NaN   4.0   NaN   NaN   4.8   NaN   NaN   4.4   NaN  ...   

                                                                      
wine_id  33154 33155 33156 33157 33158 33159 33160 33161 33162 33163  
user                                                                  
user-0     NaN   NaN   4.0   3.4   3.9   3.8   3.3   NaN   NaN   NaN  
user-1     3.6   NaN   3.4   NaN   3.8   NaN   3.9   3.9   NaN   NaN  
user-10    3.8   NaN   NaN   3.3   3.6   NaN   3.9   3.6   NaN   3.8  
user-100   3.6   NaN   NaN   3.4   3.3   3.8   NaN   3.8   NaN   3.3  
user-101   NaN   3.8   3.8   3.7   NaN   3.8   3.4   NaN   3.7   3.3  

[5 rows x 33163 columns]

In [112]:
#work in progress...

#new_ratings = ratings_pivot[5689]
#similar_to_new = ratings_pivot.corrwith(new_ratings)
#corr_new = pd.DataFrame(similar_to_new, columns=['pearsonR'])
#corr_new.dropna(inplace=True)
#corr_summary = corr_new.join(rating_counts['rating_counts'])
#corr_summary[corr_summary['rating_counts']>=300].sort_values('pearsonR', ascending=False).head(10)